In [1]:
!pip install accelerate gTTS gradio transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing i

In [3]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch
import gradio as gr
from gtts import gTTS
import os

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
language_model_name = "Qwen/Qwen2-1.5B-Instruct"

In [7]:
language_model = AutoModelForCausalLM.from_pretrained(language_model_name,trust_remote_code=True,torch_dtype=torch.float16,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(language_model_name,trust_remote_code=True)


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [8]:
language_model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [16]:
def process_input(input_text,action):
  if action== "Translate to English":
    input_text = "Translate to English: " + input_text
    lang = "en"
  elif action == "Translate to Chinese":
    input_text = "Translate to Chinese: " + input_text
    lang = "zh"
  elif action == "Translate to Tamil":
    input_text = "Translate to Tamil" + input_text
    lang = "ta"

  messages = [
      {"role":"system","content":"you are an helpful AI assistant"},
      {"role":"user","content":input_text}
  ]
  text = tokenizer.apply_chat_template(messages,tokenize=False,
        add_generation_prompt=True)
  model_inputs = tokenizer([text],return_tensors="pt").to(device)

  generated_ids = language_model.generate(
      model_inputs.input_ids,
      max_new_tokens = 512
  )
  generate_ids = [ output_ids[len(input_ids):] for input_ids,output_ids in zip(model_inputs.input_ids,generated_ids)]
  output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return output,lang





In [17]:
def text_to_speech(text,lang):
  tts = gTTS(text=text,lang=lang,slow=False)
  filename = "output.mp3"
  tts.save("output.mp3")
  return filename

In [18]:
def handle_interaction(input_text,action):
    output,lang = process_input(input_text,action)
    audio = text_to_speech(output,lang)
    return output,audio

In [19]:
action_options = ["Translate to English","Translate to Chinese","Traslate to Tamil","Chat"]

In [20]:
#gradio app

iface = gr.Interface(
    fn = handle_interaction,
    inputs = [
        gr.Textbox(label="Input Text"),
        gr.Dropdown(label="Action",choices=action_options)
    ],
    outputs = [
        gr.Textbox(label="Output Text"),
        gr.Audio(label="Output Audio")
    ],
    title = "Translate &chat App using LLM",
    theme = "gradio/soft"

)

In [21]:
if __name__ == "__main__":
    iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e172d0bb443f70c45b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
